## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,31.66,86,40,25.32,shower snow
1,1,Faya,SA,18.3851,42.4509,69.91,49,95,3.83,overcast clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,35.71,75,20,27.63,few clouds
3,3,Jamestown,US,42.0970,-79.2353,81.95,63,75,17.27,broken clouds
4,4,Namibe,AO,-15.1961,12.1522,63.36,85,1,3.29,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp = float(input("Enter max temp: "))
min_temp = float(input("Enter min temp: "))

Enter max temp: 55
Enter min temp: 50


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_temp_cities_df = city_data_df.loc[(city_data_df["Max Temp"] < max_temp) & (city_data_df["Max Temp"] > min_temp)]
preferred_temp_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Khatanga,RU,71.9667,102.5000,52.56,63,100,8.88,overcast clouds
37,37,Lebu,CL,-37.6167,-73.6500,53.53,77,91,7.07,overcast clouds
43,43,Nanortalik,GL,60.1432,-45.2371,52.34,73,5,14.50,clear sky
74,74,Bisert,RU,56.8617,59.0523,53.58,94,74,2.95,broken clouds
81,81,Gerede,TR,40.8008,32.1969,53.64,85,0,2.57,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_temp_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_temp_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Khatanga,RU,52.56,overcast clouds,71.9667,102.5000,
37,Lebu,CL,53.53,overcast clouds,-37.6167,-73.6500,
43,Nanortalik,GL,52.34,clear sky,60.1432,-45.2371,
74,Bisert,RU,53.58,broken clouds,56.8617,59.0523,
81,Gerede,TR,53.64,clear sky,40.8008,32.1969,
113,Talnakh,RU,52.21,light rain,69.4865,88.3972,
117,Batemans Bay,AU,53.10,overcast clouds,-35.7167,150.1833,
127,Mnogovershinnyy,RU,51.08,overcast clouds,53.9353,139.9242,
128,Klaksvik,FO,50.92,broken clouds,62.2266,-6.5890,
133,Melipilla,CL,54.41,clear sky,-33.7000,-71.2167,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for i, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    results = requests.get(base_url, params=params).json()
     
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[i, "Hotel Name"] = results["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = hotel_df.dropna(how='any')
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Khatanga,RU,52.56,overcast clouds,71.9667,102.5000,"""Khatanga"""
37,Lebu,CL,53.53,overcast clouds,-37.6167,-73.6500,Hostal Las Lilas
43,Nanortalik,GL,52.34,clear sky,60.1432,-45.2371,Hotel Kap Farvel
81,Gerede,TR,53.64,clear sky,40.8008,32.1969,Hotel Konuk
113,Talnakh,RU,52.21,light rain,69.4865,88.3972,Talnakh Hotel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))